In [28]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, BatchNormalization, GlobalAveragePooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder

In [29]:
TRAIN_DIR = 'C:\\Users\\praty\\Documents\\GitHub\\FaceEx\\Data\\train'
TEST_DIR = 'C:\\Users\\praty\\Documents\\GitHub\\FaceEx\\Data\\test'

In [30]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [31]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [32]:
print(train)

                                                   image     label
0      C:\Users\praty\Documents\GitHub\FaceEx\Data\tr...     angry
1      C:\Users\praty\Documents\GitHub\FaceEx\Data\tr...     angry
2      C:\Users\praty\Documents\GitHub\FaceEx\Data\tr...     angry
3      C:\Users\praty\Documents\GitHub\FaceEx\Data\tr...     angry
4      C:\Users\praty\Documents\GitHub\FaceEx\Data\tr...     angry
...                                                  ...       ...
28816  C:\Users\praty\Documents\GitHub\FaceEx\Data\tr...  surprise
28817  C:\Users\praty\Documents\GitHub\FaceEx\Data\tr...  surprise
28818  C:\Users\praty\Documents\GitHub\FaceEx\Data\tr...  surprise
28819  C:\Users\praty\Documents\GitHub\FaceEx\Data\tr...  surprise
28820  C:\Users\praty\Documents\GitHub\FaceEx\Data\tr...  surprise

[28821 rows x 2 columns]


In [33]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [34]:
print(test)

                                                  image     label
0     C:\Users\praty\Documents\GitHub\FaceEx\Data\te...     angry
1     C:\Users\praty\Documents\GitHub\FaceEx\Data\te...     angry
2     C:\Users\praty\Documents\GitHub\FaceEx\Data\te...     angry
3     C:\Users\praty\Documents\GitHub\FaceEx\Data\te...     angry
4     C:\Users\praty\Documents\GitHub\FaceEx\Data\te...     angry
...                                                 ...       ...
7061  C:\Users\praty\Documents\GitHub\FaceEx\Data\te...  surprise
7062  C:\Users\praty\Documents\GitHub\FaceEx\Data\te...  surprise
7063  C:\Users\praty\Documents\GitHub\FaceEx\Data\te...  surprise
7064  C:\Users\praty\Documents\GitHub\FaceEx\Data\te...  surprise
7065  C:\Users\praty\Documents\GitHub\FaceEx\Data\te...  surprise

[7066 rows x 2 columns]


In [35]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode="grayscale")
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [36]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [37]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [38]:
x_train = train_features/255.0
x_test = test_features/255.0

In [39]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [40]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [41]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [42]:
# Build the model
model = Sequential()

# Convolutional layers with BatchNormalization and Dropout
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(BatchNormalization())  # Normalizing the output of the conv layer
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())  # Add BatchNormalization here as well
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(GlobalAveragePooling2D())

# Flatten for fully connected layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# Output layer for 7 classes (assuming softmax for multi-class classification)
model.add(Dense(7, activation='softmax'))

c:\Users\praty\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [43]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [44]:
# Callbacks
early_stopping = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=5)

In [ ]:
model.fit(x= x_train, y = y_train, batch_size = 64, epochs = 75, validation_data = (x_test,y_test), callbacks=[early_stopping, lr_scheduler])

Epoch 1/75
451/451 ━━━━━━━━━━━━━━━━━━━━ 372s 817ms/step - accuracy: 0.2115 - loss: 2.0552 - val_accuracy: 0.2583 - val_loss: 1.8207 - learning_rate: 0.0010
Epoch 2/75
451/451 ━━━━━━━━━━━━━━━━━━━━ 373s 826ms/step - accuracy: 0.3187 - loss: 1.6816 - val_accuracy: 0.3709 - val_loss: 1.5818 - learning_rate: 0.0010
Epoch 3/75
451/451 ━━━━━━━━━━━━━━━━━━━━ 407s 901ms/step - accuracy: 0.4169 - loss: 1.5027 - val_accuracy: 0.3063 - val_loss: 1.7206 - learning_rate: 0.0010
Epoch 4/75
451/451 ━━━━━━━━━━━━━━━━━━━━ 386s 856ms/step - accuracy: 0.4608 - loss: 1.4031 - val_accuracy: 0.4843 - val_loss: 1.3648 - learning_rate: 0.0010
Epoch 5/75
451/451 ━━━━━━━━━━━━━━━━━━━━ 348s 771ms/step - accuracy: 0.4878 - loss: 1.3377 - val_accuracy: 0.5204 - val_loss: 1.2790 - learning_rate: 0.0010
Epoch 6/75
451/451 ━━━━━━━━━━━━━━━━━━━━ 313s 694ms/step - accuracy: 0.5193 - loss: 1.2709 - val_accuracy: 0.5091 - val_loss: 1.3255 - learning_rate: 0.0010
Epoch 7/75
451/451 ━━━━━━━━━━━━━━━━━━━━ 348s 772ms/step - accura